In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 100)

In [2]:
file_paths = {'in_train':'../../../data/prepared/activity_log_train.csv',
              'in_test':'../../../data/prepared/activity_log_test.csv',
              'in_missing':'../../../data/prepared/activity_log_missing.csv',
              'out_train':'../../../data/prepared/activity_log_train__ml_prepared.csv',
              'out_test':'../../../data/prepared/activity_log_test__ml_prepared.csv',
              'out_missing':'../../../data/prepared/activity_log_missing__ml_prepared.csv',
             }

df = pd.read_csv(file_paths['in_train'])

# guarantee the timestamps are datetime objects
time_columns = ['start_time','end_time']

for time_col in time_columns:
    df[time_col] = pd.to_datetime(df[time_col])

### Replace Peak Power Nulls

In [3]:
# Need to fill in the missing values for peak_20min_power.
# Use intensity * FTP to estimate the missing
filt_nan = df['peak_20min_power'].isna()

df.loc[filt_nan, 'peak_20min_power'] = df.loc[filt_nan,'ftp_power'] * df.loc[filt_nan, 'intensity']

In [4]:
cols_to_keep = ['simple_exertion','elapsed_distance','moving_time','avg_speed','ride_cruise_speed','ride_avg_power',
                'peak_20min_power', 'training_window_id']

### Replace Simple Exertion with Enumerated Categories

In [5]:
df['simple_exertion'] = df['simple_exertion'].replace({'EASY':1, 'NORMAL':2, 'HARD':3})

In [6]:
num_cols = ['elapsed_distance','moving_time','avg_speed','ride_cruise_speed','ride_avg_power','peak_20min_power']
cat_cols = ['training_window_id']
target_col = ['simple_exertion']

# 2. Pipeline Preparation

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_pipeline = Pipeline([('selector', DataFrameSelector(num_cols)),
                         ('std_scaler', StandardScaler())
                        ])

nominal_cat_pipeline = Pipeline([('selector', DataFrameSelector(cat_cols)),
                                 ('one_hot', OneHotEncoder(sparse=False))
                                ])

In [9]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[('numerical_pipeline', num_pipeline), 
                                               ('categorical_pipeline', nominal_cat_pipeline)
                                              ])

### Check Nullity 

In [10]:
df[cols_to_keep].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   simple_exertion     126 non-null    int64  
 1   elapsed_distance    126 non-null    float64
 2   moving_time         126 non-null    int64  
 3   avg_speed           126 non-null    float64
 4   ride_cruise_speed   126 non-null    float64
 5   ride_avg_power      126 non-null    float64
 6   peak_20min_power    126 non-null    float64
 7   training_window_id  126 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 8.0 KB


### Apply Pipeline to Prepare Data for ML

In [11]:
data_prepared = full_pipeline.fit_transform(df)
data_prepared.shape

(126, 19)

In [12]:
data_prepared

array([[ 0.30194675,  0.17918467,  0.50259987, ...,  0.        ,
         0.        ,  0.        ],
       [-0.5583539 , -0.7691198 ,  0.86486012, ...,  0.        ,
         0.        ,  0.        ],
       [-0.5957914 , -0.80185087,  0.88804778, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.332341  ,  0.13524432, -1.55925874, ...,  0.        ,
         0.        ,  0.        ],
       [-0.05286141,  0.17021725, -0.61943897, ...,  1.        ,
         0.        ,  0.        ],
       [-0.67087782, -0.4561569 , -1.32563783, ...,  0.        ,
         0.        ,  0.        ]])

# 3. Save Prepared Values

In [13]:
training_id_one_hot = ['training_window_'+str(x) for x in list(nominal_cat_pipeline['one_hot'].categories_[0])]
print(training_id_one_hot)

['training_window_0', 'training_window_1', 'training_window_2', 'training_window_3', 'training_window_4', 'training_window_5', 'training_window_6', 'training_window_7', 'training_window_8', 'training_window_9', 'training_window_10', 'training_window_11', 'training_window_12']


In [14]:
targets = df[target_col].values

In [16]:
column_names = num_cols + training_id_one_hot + target_col
print(column_names, len(column_names))

['elapsed_distance', 'moving_time', 'avg_speed', 'ride_cruise_speed', 'ride_avg_power', 'peak_20min_power', 'training_window_0', 'training_window_1', 'training_window_2', 'training_window_3', 'training_window_4', 'training_window_5', 'training_window_6', 'training_window_7', 'training_window_8', 'training_window_9', 'training_window_10', 'training_window_11', 'training_window_12', 'simple_exertion'] 20


In [17]:
df_prepared = pd.DataFrame(np.c_[data_prepared, targets], columns=column_names)
df_prepared.head()

,elapsed_distance,moving_time,avg_speed,ride_cruise_speed,ride_avg_power,peak_20min_power,training_window_0,training_window_1,training_window_2,training_window_3,training_window_4,training_window_5,training_window_6,training_window_7,training_window_8,training_window_9,training_window_10,training_window_11,training_window_12,simple_exertion
0,0.301947,0.179185,0.502600,-0.104944,0.559036,0.975233,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,-0.558354,-0.769120,0.864860,0.342861,0.788333,0.903988,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,-0.595791,-0.801851,0.888048,0.804365,1.269131,0.616208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0
3,-0.125927,-0.295640,0.614986,-0.118132,0.211381,0.310750,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,-1.127194,-1.180276,-0.693269,0.249776,-0.217549,-0.983908,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [18]:
df_prepared.to_csv(file_paths['out_train'], index=False)

# 4. Prepare and Save the Test Data

In [24]:
df_test = pd.read_csv(file_paths['in_test'])

In [25]:
# Need to fill in the missing values for peak_20min_power.
# Use intensity * FTP to estimate the missing
filt_nan = df_test['peak_20min_power'].isna()
df_test.loc[filt_nan, 'peak_20min_power'] = df_test.loc[filt_nan,'ftp_power'] * df_test.loc[filt_nan, 'intensity']

In [26]:
df_test['simple_exertion'] = df_test['simple_exertion'].replace({'EASY':1, 'NORMAL':2, 'HARD':3})
targets = df_test[target_col].values

In [27]:
df_test[cols_to_keep].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   simple_exertion     54 non-null     int64  
 1   elapsed_distance    54 non-null     float64
 2   moving_time         54 non-null     int64  
 3   avg_speed           54 non-null     float64
 4   ride_cruise_speed   54 non-null     float64
 5   ride_avg_power      54 non-null     float64
 6   peak_20min_power    54 non-null     float64
 7   training_window_id  54 non-null     int64  
dtypes: float64(5), int64(3)
memory usage: 3.5 KB


In [28]:
data_prepared = full_pipeline.fit_transform(df_test)
data_prepared.shape

(54, 18)

In [31]:
training_id_one_hot = ['training_window_'+str(x) for x in list(nominal_cat_pipeline['one_hot'].categories_[0])]
print(training_id_one_hot)
column_names = num_cols + training_id_one_hot + target_col
# need to ensure that training_window_0 is added as all zeros

['training_window_1', 'training_window_2', 'training_window_3', 'training_window_4', 'training_window_5', 'training_window_6', 'training_window_7', 'training_window_8', 'training_window_9', 'training_window_10', 'training_window_11', 'training_window_12']


In [32]:
df_prepared = pd.DataFrame(np.c_[data_prepared, targets], columns=column_names)
df_prepared.head()

,elapsed_distance,moving_time,avg_speed,ride_cruise_speed,ride_avg_power,peak_20min_power,training_window_1,training_window_2,training_window_3,training_window_4,training_window_5,training_window_6,training_window_7,training_window_8,training_window_9,training_window_10,training_window_11,training_window_12,simple_exertion
0,-0.583768,-0.600478,-0.009533,-0.858793,-0.328163,-0.616149,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,-0.317711,0.039052,-1.259373,-1.102532,-1.286629,-1.775306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.651350,0.480656,0.223983,1.935312,1.236084,0.202598,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,-0.182067,0.699772,-2.201339,-1.072299,-2.183943,-0.507488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
4,0.861471,0.334257,1.334372,0.176060,1.183030,0.621841,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [34]:
# Insert a training_window_0 column
training_window_0_values = np.zeros(df_prepared.shape[0])
df_prepared.insert(6, 'training_window_0', training_window_0_values)

In [36]:
df_prepared.head()

,elapsed_distance,moving_time,avg_speed,ride_cruise_speed,ride_avg_power,peak_20min_power,training_window_0,training_window_1,training_window_2,training_window_3,training_window_4,training_window_5,training_window_6,training_window_7,training_window_8,training_window_9,training_window_10,training_window_11,training_window_12,simple_exertion
0,-0.583768,-0.600478,-0.009533,-0.858793,-0.328163,-0.616149,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,-0.317711,0.039052,-1.259373,-1.102532,-1.286629,-1.775306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.651350,0.480656,0.223983,1.935312,1.236084,0.202598,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,-0.182067,0.699772,-2.201339,-1.072299,-2.183943,-0.507488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
4,0.861471,0.334257,1.334372,0.176060,1.183030,0.621841,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [37]:
df_prepared.shape

(54, 20)

In [38]:
df_prepared.to_csv(file_paths['out_test'], index=False)

# 5. Prepare and Save the Missing Dataset

In [43]:
df_missing = pd.read_csv(file_paths['in_missing'])
df_missing[cols_to_keep].head()

,simple_exertion,elapsed_distance,moving_time,avg_speed,ride_cruise_speed,ride_avg_power,peak_20min_power,training_window_id
0,NaN,5.167961,1472,13.119808,14.154069,122.304555,131.178797,0
1,NaN,5.257609,1481,13.339800,15.443823,113.161863,126.461932,0
2,NaN,7.209980,2122,12.187326,14.151900,101.118739,112.499284,1
3,NaN,8.708461,2743,11.017330,14.065320,91.116677,NaN,2
4,NaN,8.644248,2610,11.706583,14.111624,96.993258,110.849867,2


In [44]:
# Need to fill in the missing values for peak_20min_power.
# Use intensity * FTP to estimate the missing
filt_nan = df_missing['peak_20min_power'].isna()
df_missing.loc[filt_nan, 'peak_20min_power'] = df_missing.loc[filt_nan,'ftp_power'] * df_missing.loc[filt_nan, 'intensity']

In [46]:
df_missing[cols_to_keep].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   simple_exertion     0 non-null      float64
 1   elapsed_distance    38 non-null     float64
 2   moving_time         38 non-null     int64  
 3   avg_speed           38 non-null     float64
 4   ride_cruise_speed   38 non-null     float64
 5   ride_avg_power      38 non-null     float64
 6   peak_20min_power    38 non-null     float64
 7   training_window_id  38 non-null     int64  
dtypes: float64(6), int64(2)
memory usage: 2.5 KB


In [47]:
data_prepared = full_pipeline.fit_transform(df_missing)
data_prepared.shape

(38, 18)

In [48]:
training_id_one_hot = ['training_window_'+str(x) for x in list(nominal_cat_pipeline['one_hot'].categories_[0])]
print(training_id_one_hot)
column_names = num_cols + training_id_one_hot 
# need to ensure that training_window_6 is added as all zeros

['training_window_0', 'training_window_1', 'training_window_2', 'training_window_3', 'training_window_4', 'training_window_5', 'training_window_7', 'training_window_8', 'training_window_9', 'training_window_10', 'training_window_11', 'training_window_12']


In [49]:
df_prepared = pd.DataFrame(data_prepared, columns=column_names)
df_prepared.head()

,elapsed_distance,moving_time,avg_speed,ride_cruise_speed,ride_avg_power,peak_20min_power,training_window_0,training_window_1,training_window_2,training_window_3,training_window_4,training_window_5,training_window_7,training_window_8,training_window_9,training_window_10,training_window_11,training_window_12
0,-0.588873,-0.763382,1.181918,0.041841,1.300453,0.960239,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.573702,-0.759225,1.272014,0.772345,0.984611,0.812942,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.243298,-0.463130,0.800027,0.040612,0.568571,0.376923,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.010294,-0.176274,0.320864,-0.008426,0.223041,-0.290803,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.000573,-0.237710,0.603142,0.017801,0.426053,0.325416,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# Insert a training_window_0 column
training_window_6_values = np.zeros(df_prepared.shape[0])
df_prepared.insert(12, 'training_window_6', training_window_6_values)

In [51]:
df_prepared.shape

(38, 19)

In [52]:
df_prepared.to_csv(file_paths['out_missing'], index=False)